# Environment

Install environment and declare global variable

In [1]:
# base model from huggingFace or path to model
base_model = "Viet-Mistral/Vistral-7B-Chat"
adapter = "panda1835/adapter-snakegpt-vistral-7b-chat"
new_model_name = "panda1835/snakegpt-vistral-7b-chat"
dataset_name = "panda1835/rangpt-raft-chat"

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
huggingface_write_token = user_secrets.get_secret("HF_TOKEN")
wandb_key = user_secrets.get_secret("WANDB_TOKEN")

In [2]:
!pip install -qq -U accelerate peft bitsandbytes transformers trl datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import wandb
import pprint
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

2024-05-27 07:54:38.051971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 07:54:38.052075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 07:54:38.172122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Login to HuggingFace

HuggingFace is a place where we store our base model and finetuned model adapter. It also provide us libraries to work with transformer model.

In [4]:
from huggingface_hub import login
login(huggingface_write_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load the model

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)


model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token


# Build Dataset

The dataset is crawl from website of VietinBank. These are public data. Thanks Nguyen Pham Tu Anh for preparing these data. If you have others data in Vietnamese related to banking domain, please share with us.

In [ ]:
from datasets import load_dataset

rangpt_dataset = load_dataset("panda1835/rangpt-raft-chat")

In [14]:
print("Train", len(rangpt_dataset['train'].to_pandas()))
print("Test", len(rangpt_dataset['test'].to_pandas()))

Train 694
Test 174


In [22]:
system_prompt = "Bạn là một chuyên gia về rắn ở Việt Nam, nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực.\n"
system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch.\n"

def build_prompt(row, with_answer):
    context = row['instruction']
    question = row['question']
    cot_answer = row['cot_answer']

    # Remove the question appened to context
    context = context[:-len(question) - 1]

    question_with_context = prompt = f"""
<CÂU HỎI>:
{question}

<THÔNG TIN ĐƯỢC CUNG CẤP>:
{context}

Dựa trên những thông tin được cung cấp trong phần <THÔNG TIN ĐƯỢC CUNG CẤP>, hãy suy luận từng bước một để trả lời câu hỏi của người dùng được đặt ra ở <CÂU HỎI CỦA NGƯỜI DÙNG>. Làm theo các chỉ dẫn sau:
- Hãy bắt đầu suy luận với 'Bước 1'
- Kết thúc câu trả lời dưới dạng <ANSWER>: [câu trả lời của bạn]
- Nếu thông tin cung cấp không đủ để trả lời câu hỏi, hãy trả lời 'Tôi không có đủ thông tin để trả lời câu hỏi này' và dừng tại đó.

Bắt đầu câu trả lời bằng '<ANSWER>:'
            """
    
    conversation = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question_with_context},
            ]
    if with_answer:
        conversation.append({"role": "assistant", "content": cot_answer})
    return conversation

def format_chat_template(row, with_answer=True):
    conversation = build_prompt(row, with_answer)
    formatted = tokenizer.apply_chat_template(conversation, tokenize=False)
    return tokenizer(formatted)


In [ ]:
train_data = rangpt_dataset['train'].shuffle(seed=42)\
    .map(lambda x, i: format_chat_template(x), remove_columns=["type", "question", "context", "oracle_context", "cot_answer", "instruction"], with_indices=True)

In [ ]:
test_data = rangpt_dataset['test'].shuffle(seed=42)\
    .map(lambda x, i: format_chat_template(x), remove_columns=[], with_indices=True)

In [25]:
tokenizer.decode(train_data.to_pandas().loc[0, "input_ids"])

'<s><s>[INST]  <<SYS>> \nBạn là một chuyên gia về rắn ở Việt Nam, nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\nCâu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực.\nNếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch.\n\n<</SYS>> \n\n\n<CÂU HỎI>:\nRắn lục biển có bao nhiêu hàng vảy quanh thân?\n\n<THÔNG TIN ĐƯỢC CUNG CẤP>:\n<DOCUMENT>Tên Việt Nam: Rắn lục biển. Tên La tinh / Tên khoa học: Hydrophis viperina. Đặc điểm nhận dạng: Dễ dàng xác định bỡi vảy bụng của chúng, vảy bụng rộng ở phía trước và nhỏ dần ở phía sau. Hàng vảy quanh cổ 27–34; hàng vả

# Fine-Tune

In [26]:
# Set up wandb
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [27]:
import datetime
now = datetime.datetime.utcnow()

wandb.login(key = wandb_key)
run = wandb.init(
    name="vistral-rangpt-raft",
    project=f"Fine tuning vistral 7B with RanGPT data based on RAFT framework {str(now).replace(':','.')}", 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: panda1835 (abc12342). Use `wandb login --relogin` to force relogin


In [28]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [29]:
from accelerate import Accelerator
accelerator = Accelerator()
model = accelerator.prepare_model(model)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        optim="paged_adamw_32bit",
        save_steps=10,
        logging_steps=1,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_steps=3,
        group_by_length=True,
        lr_scheduler_type="constant",
        report_to="wandb",
    ),
)
model.config.use_cache = False

trainer.train(resume_from_checkpoint=False)

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.763300
2,1.815200
3,1.643900
4,1.523400
5,1.456900
6,1.352700
7,1.271000
8,1.183200
9,1.159200
10,1.103600


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

In [28]:
# Finish training 
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇███
train/grad_norm,▄▂▂▃▄▆▅▂▁▁▁▁▄▃█▃▂▁▁▂▅
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁
total_flos,3496909370720256.0
train/epoch,3.0
train/global_step,21
train/grad_norm,0.85666
train/learning_rate,0.0002
train/loss,0.1806


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(38369, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

In [ ]:
# saved_model = model.merge_and_unload()
# tokenizer.push_to_hub(adapter)
model.push_to_hub(adapter)

# Test the model

In [25]:
model.eval()
logging.set_verbosity(logging.CRITICAL)
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [43]:
test_data_df = test_data.to_pandas()
test_data_df.columns

Index(['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer',
       'instruction', 'input_ids', 'attention_mask'],
      dtype='object')

In [44]:
ft_model = AutoModelForCausalLM.from_pretrained(
    adapter,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,)

adapter_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [15]:
# Resolve this issue: https://huggingface.co/stabilityai/stable-cascade/discussions/11

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [53]:
test_idx = 3

original_model = False

stop_tokens = [tokenizer.eos_token_id, tokenizer('<|im_end|>')['input_ids'].pop()]
conversation = build_prompt(test_data_df.iloc[test_idx], with_answer=False)
formatted = tokenizer.apply_chat_template(conversation, tokenize=False) + "<|im_start|>"
    
tok = tokenizer(formatted, return_tensors="pt").to(model.device)
input_ids = tok['input_ids']

with torch.no_grad():
    if original_model:
        out_ids = model.generate(
                input_ids = input_ids,
                max_length = 32000,
                do_sample = True,
                top_p = 1.0,
                top_k = 4,
                temperature = 0.5,
                repetition_penalty = 1.0,
                eos_token_id = tokenizer.eos_token_id,
                use_cache = True,
            )
    else:
        out_ids = ft_model.generate(
                input_ids = input_ids,
                max_length = 32000,
                do_sample = True,
                top_p = 1.0,
                top_k = 4,
                temperature = 0.5,
                repetition_penalty = 1.0,
                eos_token_id = tokenizer.eos_token_id,
                use_cache = True,
            )

print("Model's answer\n", tokenizer.batch_decode(out_ids[:, input_ids.size(1): ], skip_special_tokens=True)[0].strip())   
print("-"*50)
print("Question\n", test_data_df.loc[test_idx, 'question'])
print("-"*50)
print("Context\n", test_data_df.loc[test_idx, 'context']['sentences'])
print("-"*50)
print("Groundtruth\n", test_data_df.loc[test_idx, 'cot_answer'])
print("-"*50)
print("Input\n", tokenizer.decode(input_ids[0]))

Model's answer
 Bước 1: Xác định các đặc điểm sinh học liên quan đến cách sinh sản của rắn.

Bước 2: Xác định đặc điểm sinh học nào quyết định đến cách sinh sản của rắn.

**##begin_quote##Tuỳ theo mỗi giống rắn mà chúng sẽ có đặc điểm sinh học riêng biệt phục vụ cho hành vi sinh sản đặc thù của bản thân - là đẻ trứng hoặc đẻ con.##end_quote##

Bước 3: Trả lời câu hỏi dựa trên các đặc điểm sinh học đã xác định.

<ANSWER>: Tuỳ theo mỗi giống rắn mà chúng sẽ có đặc điểm sinh học riêng biệt phục vụ cho hành vi sinh sản đặc thù của bản thân - là đẻ trứng hoặc đẻ con.
--------------------------------------------------
Question
 Đặc điểm sinh học của rắn nào quyết định đến cách sinh sản của chúng?
--------------------------------------------------
Context
 [array(['Quản lý hoặc kiểm soát quần thể rắn xâm lấn như thế nào? - Các quần thể rắn xâm lấn sẽ được các cơ quan có thẩm quyền xử lý bằng việc di dời vào các tổ chức cứu hộ hoặc tiêu huỷ bằng biện pháp an tử?',
        'Rắn đẻ trứng (đẻ trứ

In [32]:
import torch

# Clear GPU memory cache
torch.cuda.empty_cache()
